In [ ]:
# @title 1. Setup, Load Data & Cleaning
!pip install -q transformers datasets evaluate scikit-learn accelerate

import os
import re
import time
import psutil
import torch
import numpy as np
import pandas as pd
from google.colab import drive
from datasets import load_dataset
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from scipy.special import softmax

# 1. Mount Drive
drive.mount('/content/drive')
SAVE_PATH = '/content/drive/My Drive/SLM_Research/IMDB_BERT_base'
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. Load Dataset (IMDB)
print("--- Loading Stanford IMDB Dataset ---")
dataset = load_dataset("imdb")

# 3. Clean Text (Chuẩn hóa: Xóa URL, HTML, Lowercase)
def clean_text(example):
    text = example['text']
    text = text.lower()
    # Xóa URL
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Xóa HTML
    text = re.sub(r'<br\s*/>', ' ', text)
    # Xóa khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text).strip()
    example['text'] = text
    return example

print("--- Cleaning Dataset (Removing URLs & HTML) ---")
dataset = dataset.map(clean_text)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00
Mounted at /content/drive
Using device: cuda
--- Loading Stanford IMDB Dataset ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

...

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

--- Cleaning Dataset (Removing URLs & HTML) ---


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# @title 2. Tokenization & Data Splits for BERT-base

# 1. Load Tokenizer
MODEL_NAME = 'bert-base-uncased'
print(f"--- Loading Tokenizer: {MODEL_NAME} ---")
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

# 2. Tokenize Function
def tokenize_function(examples):
    # Max length 512, Truncation, Padding
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

print("--- Tokenizing Dataset ---")
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 3. Format PyTorch
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

# 4. Split Train/Val (90/10)
print("--- Splitting Train set into Train/Val ---")
train_val_split = tokenized_datasets["train"].train_test_split(test_size=0.1, seed=42)
dataset_train = train_val_split["train"]
dataset_val = train_val_split["test"]
dataset_test = tokenized_datasets["test"]

print(f"Dataset ready: Train({len(dataset_train)}), Val({len(dataset_val)}), Test({len(dataset_test)})")

--- Loading Tokenizer: bert-base-uncased ---


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

--- Tokenizing Dataset ---


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

--- Splitting Train set into Train/Val ---
Dataset ready: Train(22500), Val(2500), Test(25000)


In [ ]:
# @title 3. Training BERT-base (Golden Config)

# 1. Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    probs = softmax(logits, axis=1)[:, 1]

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    roc_auc = roc_auc_score(labels, probs)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall, 'roc_auc': roc_auc}

# 2. Model Init
print(f"--- Loading Model: {MODEL_NAME} ---")
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)

# 3. Training Arguments (Giữ nguyên không đổi)
training_args = TrainingArguments(
    output_dir='./results_bert_base',
    num_train_epochs=5,              # 5 Epochs
    per_device_train_batch_size=16,  # Batch 16
    per_device_eval_batch_size=32,   # Batch 32
    learning_rate=5e-5,              # LR 5e-5
    warmup_steps=500,                # Warmup 500
    weight_decay=0.01,               # Decay 0.01
    logging_dir='./logs_bert_base',
    logging_steps=100,

    # --- Chiến lược Epoch ---
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2,
    fp16=True,                       # Mixed Precision
    report_to="none"
)

# 4. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# 5. Start Training
print("--- Starting Training (BERT-base) ---")
start_train_time = time.time()
trainer.train()
end_train_time = time.time()

training_time = end_train_time - start_train_time
print(f"\nTraining completed in: {training_time:.2f} seconds")

# 6. Save Model
print(f"Saving model to {SAVE_PATH}...")
trainer.save_model(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)
print("BERT-base Model saved successfully!")

--- Loading Model: bert-base-uncased ---


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--- Starting Training (BERT-base) ---


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Roc Auc
1,0.239700,0.275647,0.912400,0.907712,0.965054,0.856802,0.974013
2,0.166900,0.230012,0.937200,0.936049,0.959098,0.914081,0.980282


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Roc Auc
1,0.239700,0.275647,0.912400,0.907712,0.965054,0.856802,0.974013
2,0.166900,0.230012,0.937200,0.936049,0.959098,0.914081,0.980282
3,0.091800,0.316274,0.933600,0.932956,0.947498,0.918854,0.979660
4,0.044800,0.344792,0.936000,0.935327,0.950698,0.920446,0.980009
5,0.004300,0.388121,0.934800,0.934512,0.943994,0.925219,0.978764



Training completed in: 2830.36 seconds
Saving model to /content/drive/My Drive/SLM_Research/IMDB_BERT_base...
BERT-base Model saved successfully!


In [ ]:
# @title 4. Final Evaluation on Test Set (Robust Version)
import os
import time
import psutil
import torch
import pandas as pd

print("--- Running Evaluation on Test Set ---")

# Kiểm tra các biến bắt buộc
if 'trainer' not in locals() or 'dataset_test' not in locals():
    raise ValueError("Lỗi: Biến 'trainer' hoặc 'dataset_test' chưa được định nghĩa.")

# 1. Classification Metrics
start_pred_time = time.time()
predictions_output = trainer.predict(dataset_test)
end_pred_time = time.time()
metrics = predictions_output.metrics

# 2. Efficiency Metrics
total_samples = len(dataset_test)
total_inference_time = end_pred_time - start_pred_time
latency_per_sample = (total_inference_time / total_samples) * 1000 # ms

# 3. Model Size Check
safetensors_path = os.path.join(SAVE_PATH, 'model.safetensors')
bin_path = os.path.join(SAVE_PATH, 'pytorch_model.bin')

if os.path.exists(safetensors_path):
    model_size = os.path.getsize(safetensors_path) / (1024 * 1024)
    format_type = "safetensors"
elif os.path.exists(bin_path):
    model_size = os.path.getsize(bin_path) / (1024 * 1024)
    format_type = "bin"
else:
    model_size = 0
    format_type = "unknown"

# Handle training_time variable
current_training_time = training_time if 'training_time' in locals() else 0.0

# Resource Usage
process = psutil.Process(os.getpid())
ram_usage = process.memory_info().rss / 1024 ** 2
vram_usage = torch.cuda.memory_allocated() / 1024 ** 2 if torch.cuda.is_available() else 0

# 4. Report
print("\n====== REPORT: BERT-base on IMDB ======")
print(f"1. Classification Metrics:")
print(f"   - Accuracy:  {metrics.get('test_accuracy', 0):.4f}")
print(f"   - F1-Score:  {metrics.get('test_f1', 0):.4f}")
print(f"   - ROC-AUC:   {metrics.get('test_roc_auc', 0):.4f}")

print(f"\n2. Efficiency Metrics:")
print(f"   - Training Time:      {current_training_time:.2f} s")
print(f"   - Inference Latency:  {latency_per_sample:.4f} ms/sample")
print(f"   - Model Size ({format_type}): {model_size:.2f} MB")
print(f"   - Peak RAM Usage:     {ram_usage:.2f} MB")
print(f"   - Peak VRAM Usage:    {vram_usage:.2f} MB")

# 5. Save CSV
results_df = pd.DataFrame({
    "Metric": ["Accuracy", "F1", "ROC-AUC", "Training Time (s)", "Inference Latency (ms)", "Model Size (MB)"],
    "Value": [
        metrics.get('test_accuracy', 0),
        metrics.get('test_f1', 0),
        metrics.get('test_roc_auc', 0),
        current_training_time,
        latency_per_sample,
        model_size
    ]
})
results_file = os.path.join(SAVE_PATH, 'imdb_bert_base_results.csv')
results_df.to_csv(results_file, index=False)
print(f"\nReport saved to {results_file}")

--- Running Evaluation on Test Set ---



====== REPORT: BERT-base on IMDB ======
1. Classification Metrics:
   - Accuracy:  0.9389
   - F1-Score:  0.9381
   - ROC-AUC:   0.9840

2. Efficiency Metrics:
   - Training Time:      2830.36 s
   - Inference Latency:  6.7675 ms/sample
   - Model Size (safetensors): 417.67 MB
   - Peak RAM Usage:     2414.50 MB
   - Peak VRAM Usage:    1278.29 MB

Report saved to /content/drive/My Drive/SLM_Research/IMDB_BERT_base/imdb_bert_base_results.csv


In [3]:
# @title LOAD & TEST: BERT-base (Full Metrics - No Retrain)
!pip install -q transformers datasets evaluate scikit-learn accelerate

import os
import time
import psutil
import torch
import pandas as pd
import numpy as np
import re
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from scipy.special import softmax
from google.colab import drive

# 1. Setup & Path
drive.mount('/content/drive')

# --- CẤU HÌNH ĐƯỜNG DẪN (Hãy kiểm tra lại folder bạn đã lưu BERT-base) ---
# Thường là một trong các đường dẫn này, bạn hãy sửa lại nếu khác
SAVE_PATH = '/content/drive/My Drive/SLM_Research/IMDB_BERT_base'
# hoặc: SAVE_PATH = '/content/drive/My Drive/SLM_Research/IMDB_BERT_Full_FT'

if not os.path.exists(SAVE_PATH):
    print(f"CẢNH BÁO: Không tìm thấy đường dẫn {SAVE_PATH}")
    print("Hãy kiểm tra lại tên folder bạn đã lưu model BERT-base.")
else:
    print(f"Đã tìm thấy model tại: {SAVE_PATH}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. Load Data & Tokenizer
print("--- Loading Data ---")
dataset = load_dataset("imdb")

def clean_text(example):
    text = example['text']
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'<br\s*/>', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    example['text'] = text
    return example

dataset = dataset.map(clean_text)

# Load Tokenizer từ folder đã lưu (để đảm bảo khớp với lúc train)
try:
    tokenizer = AutoTokenizer.from_pretrained(SAVE_PATH)
except:
    print("Không tìm thấy tokenizer trong folder model, load tokenizer gốc.")
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

print("--- Tokenizing Test Set ---")
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
dataset_test = tokenized_datasets["test"]

# 3. Load Saved Model
print(f"--- Loading Saved Model from {SAVE_PATH} ---")
model = AutoModelForSequenceClassification.from_pretrained(SAVE_PATH)
model.to(device)

# 4. Evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    probs = softmax(logits, axis=1)[:, 1]

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    roc_auc = roc_auc_score(labels, probs)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall, 'roc_auc': roc_auc}

trainer = Trainer(model=model, compute_metrics=compute_metrics)

print("--- Running Evaluation on Test Set ---")
start_pred_time = time.time()
predictions_output = trainer.predict(dataset_test)
end_pred_time = time.time()

# 5. Calculate Metrics
metrics = predictions_output.metrics

# Efficiency
total_samples = len(dataset_test)
total_inference_time = end_pred_time - start_pred_time
latency_per_sample = (total_inference_time / total_samples) * 1000 # ms

# Model Size
safetensors_path = os.path.join(SAVE_PATH, 'model.safetensors')
bin_path = os.path.join(SAVE_PATH, 'pytorch_model.bin')

if os.path.exists(safetensors_path):
    model_size = os.path.getsize(safetensors_path) / (1024 * 1024)
    format_type = "safetensors"
elif os.path.exists(bin_path):
    model_size = os.path.getsize(bin_path) / (1024 * 1024)
    format_type = "bin"
else:
    model_size = 0
    format_type = "unknown"

# RAM Usage
process = psutil.Process(os.getpid())
ram_usage = process.memory_info().rss / 1024 ** 2
vram_usage = torch.cuda.memory_allocated() / 1024 ** 2 if torch.cuda.is_available() else 0

# 6. Report & Save
print("\n====== REPORT: BERT-base (Full Fine-Tuning) ======")
print(f"1. Classification Metrics:")
print(f"   - Accuracy:  {metrics.get('test_accuracy', 0):.4f}")
print(f"   - Precision: {metrics.get('test_precision', 0):.4f}")
print(f"   - Recall:    {metrics.get('test_recall', 0):.4f}")
print(f"   - F1-Score:  {metrics.get('test_f1', 0):.4f}")
print(f"   - ROC-AUC:   {metrics.get('test_roc_auc', 0):.4f}")

print(f"\n2. Efficiency Metrics:")
print(f"   - Inference Latency:  {latency_per_sample:.4f} ms/sample")
print(f"   - Model Size ({format_type}): {model_size:.2f} MB")
print(f"   - Peak RAM Usage:     {ram_usage:.2f} MB")
print(f"   - Peak VRAM Usage:    {vram_usage:.2f} MB")

results_df = pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1", "ROC-AUC", "Inference Latency (ms)", "Model Size (MB)"],
    "Value": [
        metrics.get('test_accuracy', 0),
        metrics.get('test_precision', 0),
        metrics.get('test_recall', 0),
        metrics.get('test_f1', 0),
        metrics.get('test_roc_auc', 0),
        latency_per_sample,
        model_size
    ]
})
results_file = os.path.join(SAVE_PATH, 'imdb_bert_base_results_full.csv')
results_df.to_csv(results_file, index=False)
print(f"\nReport saved to {results_file}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Đã tìm thấy model tại: /content/drive/My Drive/SLM_Research/IMDB_BERT_base
Using device: cuda
--- Loading Data ---


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

--- Tokenizing Test Set ---


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

wandb: Enter your choice:

wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"



====== REPORT: BERT-base (Full Fine-Tuning) ======
1. Classification Metrics:
   - Accuracy:  0.9388
   - Precision: 0.9497
   - Recall:    0.9267
   - F1-Score:  0.9381
   - ROC-AUC:   0.9840

2. Efficiency Metrics:
   - Inference Latency:  33.6007 ms/sample
   - Model Size (safetensors): 417.67 MB
   - Peak RAM Usage:     2338.55 MB
   - Peak VRAM Usage:    427.86 MB

Report saved to /content/drive/My Drive/SLM_Research/IMDB_BERT_base/imdb_bert_base_results_full.csv
